In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df=pd.read_csv("C:/Machine Learning Projects/Fraud Detection using Machine Learning/data/Imported_data.csv")
display(df.head())      ## (or) df.head() This will display the first 5 rows of the dataset


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [2]:
df.duplicated().sum()

0

In [31]:
df.columns = df.columns.str.strip()
df.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')

In [32]:
num_col=df.select_dtypes(exclude=['object']).columns
num_col

Index(['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest',
       'newbalanceDest', 'isFraud', 'isFlaggedFraud'],
      dtype='object')

In [2]:
df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

    Step 1: Handling Missing Values:
Impute missing values (e.g., mean, median, mode, or using advanced techniques like KNN).

_**In our scenario we don't have any missing values in the dataset**_

    Step 2. Encoding Categorical Variables:

Apply methods like one-hot encoding, label encoding, or target encoding.

_**We will perform the data encoding steps below**_

In [33]:
from sklearn.preprocessing import OneHotEncoder

# Initialize OneHotEncoder with drop='first' and sparse_output=False
encoder = OneHotEncoder(drop='first', sparse_output=False)

# Fit and transform the 'type' column
encoded = encoder.fit_transform(df[['type']])

# Get the feature names for the encoded columns
feature_names = encoder.get_feature_names_out(['type'])

# Add the encoded columns directly to the original DataFrame
df[feature_names] = encoded

# Drop the original 'type' column
df.drop(columns=['type'], inplace=True)

# Display the updated DataFrame
df.head()

,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0,0.0,0.0,1.0,0.0
1,1,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0,0.0,0.0,1.0,0.0
2,1,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0,0.0,0.0,0.0,1.0
3,1,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0,1.0,0.0,0.0,0.0
4,1,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0,0.0,0.0,1.0,0.0


### Observations:

1. One-hot encoding was applied to the categorical variable `type` to convert it into a numerical format suitable for machine learning models.

2. The categorical variables `nameOrig` (sender account name) and `nameDest` (recipient account name) were determined that columns do not provide significant insights for identifying fraudulent transactions.

3. These columns were excluded from further analysis to:

 - Reduce noise in the dataset.
 - Enhance model performance by focusing on more relevant features.


In [34]:
df.head()

,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0,0.0,0.0,1.0,0.0
1,1,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0,0.0,0.0,1.0,0.0
2,1,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0,0.0,0.0,0.0,1.0
3,1,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0,1.0,0.0,0.0,0.0
4,1,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0,0.0,0.0,1.0,0.0


In [35]:
df.drop(columns=['nameOrig', 'nameDest'], inplace=True)

    Step 3. Feature Transformation/Handling Outliers:
Apply transformations like log, square root, or power transformations to normalize or reduce skewness.

__Rule of Thumb:__
 - Skewness between -0.5 and 0.5: No transformation is usually needed.
 - Skewness less than -1 or greater than 1: Transformation is likely required.

In [36]:
# Check skewness of the 'amount' column
amount_skewness = df['amount'].skew()
print(f"Skewness of 'amount' before: {amount_skewness}")

# Apply log transformation
# df['amount'] = np.log1p(df['amount'])  # np.log1p ensures no error for zero values
# print(f"Skewness of transformed 'amount after log transformation ': {df['amount'].skew()}")

Skewness of 'amount' before: 30.99394948249038


In [37]:
step_skewness = df['step'].skew()
print(f"Skewness of 'step': {step_skewness}")

Skewness of 'step': 0.37517688846984765


_**Creating a function which transforms if the data skewness is less/greater than threshold**_

In [38]:
import numpy as np

def apply_transformation_based_on_skew(df, columns, skew_thresholds=(-0.5, 0.5)):
    for col in columns:
        skew_before = df[col].skew()
        
        # Print skewness before transformation
        print(f"Skewness of '{col}' before: {skew_before}")
        
        # Check if skewness is within the specified threshold range
        if skew_before < skew_thresholds[0] or skew_before > skew_thresholds[1]:
            # Apply log transformation if skewness is outside the acceptable range
            df[col] = np.log1p(df[col])
            skew_after = df[col].skew()
            print(f"Skewness of '{col}' after transformation: {skew_after}")
        else:
            print(f"Skewness of '{col}' is within acceptable range, no transformation applied.")
            
# Define the columns to apply transformations to
columns = ['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']

# Call the function with the dataframe (df) and column list
apply_transformation_based_on_skew(df, columns, skew_thresholds=(-0.5, 0.5))


Skewness of 'step' before: 0.37517688846984765
Skewness of 'step' is within acceptable range, no transformation applied.
Skewness of 'amount' before: 30.99394948249038
Skewness of 'amount' after transformation: -0.5550183204227738
Skewness of 'oldbalanceOrg' before: 5.249136420603121
Skewness of 'oldbalanceOrg' after transformation: -0.25972282464865
Skewness of 'newbalanceOrig' before: 5.176884001159233
Skewness of 'newbalanceOrig' after transformation: 0.44014809990739817
Skewness of 'oldbalanceDest' before: 19.921757915791062
Skewness of 'oldbalanceDest' after transformation: -0.21326075958033924
Skewness of 'newbalanceDest' before: 19.352302057660165
Skewness of 'newbalanceDest' after transformation: -0.3869372982497443


    Step 4. Feature Scaling:
        
Scale features using techniques like Min-Max scaling, Standardization (Z-score), or Robust scaling.


In [39]:
df.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,1,9.194276,12.044359,11.984786,0.000000,0.0,0,0,0.0,0.0,1.0,0.0
1,1,7.531166,9.964112,9.872292,0.000000,0.0,0,0,0.0,0.0,1.0,0.0
2,1,5.204007,5.204007,0.000000,0.000000,0.0,1,0,0.0,0.0,0.0,1.0
3,1,5.204007,5.204007,0.000000,9.960954,0.0,1,0,1.0,0.0,0.0,0.0
4,1,9.364703,10.634773,10.305174,0.000000,0.0,0,0,0.0,0.0,1.0,0.0


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   amount          float64
 2   oldbalanceOrg   float64
 3   newbalanceOrig  float64
 4   oldbalanceDest  float64
 5   newbalanceDest  float64
 6   isFraud         int64  
 7   isFlaggedFraud  int64  
 8   type_CASH_OUT   float64
 9   type_DEBIT      float64
 10  type_PAYMENT    float64
 11  type_TRANSFER   float64
dtypes: float64(9), int64(3)
memory usage: 582.5 MB


In [42]:
from sklearn.preprocessing import RobustScaler

# Create an instance of RobustScaler
scaler = RobustScaler()

# List of columns to scale (make sure this is a flat list)
columns = ['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']

# Apply RobustScaler to the selected columns and directly update the dataframe
df[columns] = scaler.fit_transform(df[columns])

# Now df is updated with the scaled values, you can check it directly
df.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,-1.329609,-0.738870,0.214333,1.008874,-0.857455,-0.881855,0,0,0.0,0.0,1.0,0.0
1,-1.329609,-1.344417,0.034746,0.831045,-0.857455,-0.881855,0,0,0.0,0.0,1.0,0.0
2,-1.329609,-2.191750,-0.376191,0.000000,-0.857455,-0.881855,1,0,0.0,0.0,0.0,1.0
3,-1.329609,-2.191750,-0.376191,0.000000,-0.133384,-0.881855,1,0,1.0,0.0,0.0,0.0
4,-1.329609,-0.676816,0.092644,0.867485,-0.857455,-0.881855,0,0,0.0,0.0,1.0,0.0


_**Observations**_

1. The dataset contains several outliers, specifically in the variables: _**amount**_, _**oldbalanceOrg**_, _**newbalanceOrig**_, _**oldbalanceDest**_, _**newbalanceDest**_

2. Since the data points represent currency transactions, performing outlier analysis might not be appropriate. These transactions reflect individual interests and variations, which are expected and inherent in financial data.

3. Given this context, we will use the __RobustScaler__, which is less sensitive to outliers and will scale the data effectively without being influenced by extreme values.

    Step 5. Creating New Features:
Derive new features from existing ones (e.g., ratios, differences, or domain-specific combinations).

However, at this stage, there is no immediate need to create additional features, as the current set of features is sufficient for the analysis.